In [28]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [29]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

In [35]:
class Dense(layers.Layer):
    def __init__(self,units,input_dim):
        super(Dense,self).__init__()
        self.w=self.add_weight(
            name="w", shape=(input_dim,units),
            initializer="random_normal", trainable=True,
        )

        self.b=self.add_weight(
            name="b", shape=(units,),
            initializer="zeros", trainable=True,
        )

    def call(self,inputs):
        return tf.matmul(inputs,self.w)+self.b

In [36]:
class MyModel(keras.Model):
    def __init__(self,num_classes=10):
        super(MyModel,self).__init__()
        self.dense1=Dense(64,784)
        self.dense2=Dense(10,64)
    def call(self,input_tensor):
        x=tf.nn.relu(self.dense1(input_tensor))
        return self.dense2(x)

In [37]:
model=MyModel()

In [38]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=.001),
    metrics=["accuracy"],
)

In [39]:
model.fit(x_train,y_train,batch_size=32,epochs=5,verbose=2)

Epoch 1/5
1875/1875 - 3s - loss: 0.3391 - accuracy: 0.9055 - 3s/epoch - 2ms/step
Epoch 2/5
1875/1875 - 2s - loss: 0.1550 - accuracy: 0.9544 - 2s/epoch - 1ms/step
Epoch 3/5
1875/1875 - 3s - loss: 0.1125 - accuracy: 0.9665 - 3s/epoch - 1ms/step
Epoch 4/5
1875/1875 - 2s - loss: 0.0886 - accuracy: 0.9733 - 2s/epoch - 1ms/step
Epoch 5/5
1875/1875 - 3s - loss: 0.0733 - accuracy: 0.9779 - 3s/epoch - 1ms/step


In [40]:
print(model.summary())

Model: "my_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             multiple                  50240     
                                                                 
 dense_9 (Dense)             multiple                  650       
                                                                 
Total params: 50890 (198.79 KB)
Trainable params: 50890 (198.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [41]:
model.evaluate(x_test,y_test,batch_size=32,verbose=2)

313/313 - 0s - loss: 0.0886 - accuracy: 0.9713 - 470ms/epoch - 2ms/step


[0.0886196568608284, 0.9713000059127808]

Now notice here, that in this way we need to initialize the inputshpe of a layer. But in default system we only need to specify the units. To make our custom layer to work like this, we need to define a build function inside the Dense class. We will see this in the next notebook.